## models

- `paraphrase-MiniLM-L6-v2`
    - embedding dimension：384 = 32*12

```
from sentence_transformers import SentenceTransformer, util
embed_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embed_1 = embed_model.encode(sentence1, convert_to_tensor=True)
embed_2 = embed_model.encode(sentence2, convert_to_tensor=True)
cos_sim = util.pytorch_cos_sim(embed_1, embed_2).item()
```

In [6]:
from sentence_transformers import SentenceTransformer, util
embed_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embed_1 = embed_model.encode('the movie is great!', convert_to_tensor=True)
embed_2 = embed_model.encode('positive', convert_to_tensor=True)
cos_sim = util.pytorch_cos_sim(embed_1, embed_2).item()
print(cos_sim)
embed_3 = embed_model.encode('negative', convert_to_tensor=True)
cos_sim = util.pytorch_cos_sim(embed_1, embed_3).item()
print(cos_sim)

0.1806926727294922
-0.12082745879888535
